In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16  # how many independent sequences will we process in parallel?
block_size = 32  # what is the maximum context length for predictions?
epochs = 5000
eval_interval = 100
learning_rate = 1e-3
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

vocab_size = 65

torch.manual_seed(1337)

In [2]:
# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

chars = sorted(list(set(text)))
# vocab_size = len(chars)

In [3]:
# create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [
    stoi[c] for c in s
]  # encoder: take a string, output a list of integers

decode = lambda l: "".join(
    [itos[i] for i in l]
)  # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [4]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


# a, b = get_batch("train")
# print(a.shape, b.shape)
# print(a[0])
# print(b[0])

In [5]:
class Head(nn.Module):
    """one head of self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)  # (B,T,C)
        q = self.query(x)  # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**-0.5  # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B,T,C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedFoward(nn.Module):
    """a simple linear layer followed by a non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [6]:
class Embedding(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T,C)
        return tok_emb + pos_emb  # (B,T,C)


class Output(nn.Module):
    def __init__(self):
        super().__init__()
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, x, targets=None):
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        return logits

In [7]:
class FinalModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.a = Embedding()
        self.blocks = nn.Sequential(
            *[Block(n_embd, n_head=n_head) for _ in range(n_layer)]
        )
        self.b = Output()

    def forward(self, x, targets=None):
        x = self.a(x)  # (B,T,C)

        x = self.blocks(x)

        logits = self.b(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

In [8]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


model = FinalModel()
m = model.to(device)

print(sum(p.numel() for p in m.parameters()) / 1e6, "M parameters")

for i, j in model.state_dict().items():
    print(i, j.shape)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

0.209729 M parameters
a.token_embedding_table.weight torch.Size([65, 64])
a.position_embedding_table.weight torch.Size([32, 64])
blocks.0.sa.heads.0.tril torch.Size([32, 32])
blocks.0.sa.heads.0.key.weight torch.Size([16, 64])
blocks.0.sa.heads.0.query.weight torch.Size([16, 64])
blocks.0.sa.heads.0.value.weight torch.Size([16, 64])
blocks.0.sa.heads.1.tril torch.Size([32, 32])
blocks.0.sa.heads.1.key.weight torch.Size([16, 64])
blocks.0.sa.heads.1.query.weight torch.Size([16, 64])
blocks.0.sa.heads.1.value.weight torch.Size([16, 64])
blocks.0.sa.heads.2.tril torch.Size([32, 32])
blocks.0.sa.heads.2.key.weight torch.Size([16, 64])
blocks.0.sa.heads.2.query.weight torch.Size([16, 64])
blocks.0.sa.heads.2.value.weight torch.Size([16, 64])
blocks.0.sa.heads.3.tril torch.Size([32, 32])
blocks.0.sa.heads.3.key.weight torch.Size([16, 64])
blocks.0.sa.heads.3.query.weight torch.Size([16, 64])
blocks.0.sa.heads.3.value.weight torch.Size([16, 64])
blocks.0.sa.proj.weight torch.Size([64, 64])
bl

In [9]:
for iter in range(epochs):
    if iter % eval_interval == 0 or iter == epochs - 1:
        losses = estimate_loss()
        print(
            f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}"
        )

    xb, yb = get_batch("train")

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5091, val loss 2.5060
step 300: train loss 2.4199, val loss 2.4338
step 400: train loss 2.3499, val loss 2.3561
step 500: train loss 2.2965, val loss 2.3129
step 600: train loss 2.2405, val loss 2.2495
step 700: train loss 2.2053, val loss 2.2194
step 800: train loss 2.1625, val loss 2.1853
step 900: train loss 2.1248, val loss 2.1513
step 1000: train loss 2.1035, val loss 2.1305
step 1100: train loss 2.0709, val loss 2.1193
step 1200: train loss 2.0375, val loss 2.0789
step 1300: train loss 2.0246, val loss 2.0646
step 1400: train loss 1.9936, val loss 2.0371
step 1500: train loss 1.9720, val loss 2.0318
step 1600: train loss 1.9629, val loss 2.0479
step 1700: train loss 1.9422, val loss 2.0139
step 1800: train loss 1.9096, val loss 1.9963
step 1900: train loss 1.9080, val loss 1.9884
step 2000: train loss 1.8837, val loss 1.9943
step 2100: train loss 1.8713, val loss 1.9749


In [10]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


Flies.
Why, the have hudy to
Twiter arts the muscue: I mile
Your TI sea it; and says, and for one that I have of this Matritight.

CORIOLANUS:
Well, them not, metrieds thou shall diesty rege;
Was way is your name Borry eyre the give bagener at, though it my mily.

POF AUMKERLANE:
Upass:
You uow to, shide spost I vour I man? this world sappy:
Un in agarte is woffacer time in may.

BENVOLIO:
A thas now, stold's bewis'd your.

TRATCLAREY:
Shall
My sworn march, farwit,
As cousinst: I state, those make me,
And fliess, why his my wontly all
my nourt all upon entrram, that not
Is apon arth,
Her me and why compor?'

EDWORDIUS:
If your tarrue wells
As but hast thou beatl now thirge:
It widl-sweet you, I'll brieing like a bettly dreeness of Tyberrought thou thront,
Als troarish, I id I knom her
Where promile. HaNY we lase; my not thou
It that rige son!
Thee will, contate lips and my lours
What mineback he is all that tho sofors of that not,
Fhe what I stray mity whre sting,
that the with thou a

In [11]:
from pathlib import Path

# 1. Create models directory
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path
MODEL_NAME = "shakespeare-transformer.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(
    obj=model.state_dict(),  # only saving the state_dict() only saves the models learned parameters
    f=MODEL_SAVE_PATH,
)

Saving model to: models/shakespeare-transformer.pth
